<a href="https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/homeworks/nlp-for-text-adventures/NLP_for_Text_Adventure_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP For Text Adventure Games

# WordNet 
[WordNet](https://wordnet.princeton.edu) is a lexical knowledge base that encodes a ton of useful information about how words relate to each other.  NLTK provides a Python API to WordNet.

In [0]:
#!sudo pip3 install nltk 
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Word Senses
Words with multiple meanings are called _polysemous_ words.  An example of a polysemous word is the word _bug_ which can mean 
1. an insect
2. a virus or microbe that makes you sick
3. an error in your computer program
4. a covert listening device
5. (verb) to annoy/bother
6. (verb) to wiretap

WordNet oraganizes word senses into a structure called _synsets_. Each word can have multiple synsets, and each synset represents a different meaning of the word.

In [0]:
def get_senses(word):
  """Returns a list of word senses (WordNet synsets) for a word"""
  word_senses = wn.synsets(word)
  return word_senses

def get_definition(word_sense):
  return word_sense.definition()

def get_synonyms(word_sense):
  synonyms = []
  for lemma in word_sense.lemmas():
    synonym = lemma.name().replace('_', ' ')
    synonyms.append(synonym)
  return synonyms

# Here are the word senses for "bug" (Chris's favorite polysemous word)
# We can see what their distinct meanings are by getting their definitions or 
# their synonyms from WordNet.

word_senses = get_senses("bug")
for i, word_sense in enumerate(word_senses):
  print("\nSense %d: %s" % (i, word_sense.name()))
  print("Definition: ", get_definition(word_sense))
  print("Synonyms: ", get_synonyms(word_sense))


Sense 0: bug.n.01
Definition:  general term for any insect or similar creeping or crawling invertebrate
Synonyms:  ['bug']

Sense 1: bug.n.02
Definition:  a fault or defect in a computer program, system, or machine
Synonyms:  ['bug', 'glitch']

Sense 2: bug.n.03
Definition:  a small hidden microphone; for listening secretly
Synonyms:  ['bug']

Sense 3: hemipterous_insect.n.01
Definition:  insects with sucking mouthparts and forewings thickened and leathery at the base; usually show incomplete metamorphosis
Synonyms:  ['hemipterous insect', 'bug', 'hemipteran', 'hemipteron']

Sense 4: microbe.n.01
Definition:  a minute life form (especially a disease-causing bacterium); the term is not in technical use
Synonyms:  ['microbe', 'bug', 'germ']

Sense 5: tease.v.01
Definition:  annoy persistently
Synonyms:  ['tease', 'badger', 'pester', 'bug', 'beleaguer']

Sense 6: wiretap.v.01
Definition:  tap a telephone or telegraph wire to get information
Synonyms:  ['wiretap', 'tap', 'intercept', 'bug

## Hypernyms / Hyponyms

In addition to representing word senses, WordNet also organizes words hierachically. For example, _red_ is a specific kind of _color_, or _microbe_ is a kind of _organism_.  These are example of _hyponym_ relationships.  If X is-a Y then X is a hyponym of Y, and Y is a hypernym of X. So _red_ is a hyponym of _color_ and _color_ is a hypernym of _red_.

In WordNet, each word sense (synset) has its own distinct hypernyms and hyponyms. 

In [0]:
hyper = lambda s: s.hypernyms()
hypo = lambda s: s.hyponyms()

def get_hypernyms(word_sense, depth=5):
  return list(word_sense.closure(hyper, depth=depth))

def get_hyponyms(word_sense, depth=5):
  return list(word_sense.closure(hypo, depth=depth))

word_senses = get_senses("bug")
for i, word_sense in enumerate(word_senses):
  # The synset names include a word from the set of synonyms, 
  # plus a part of speech (n for noun, v for verb), and 
  # the number of the sense (sense 01 is the most common sense).
  print("\nSense %d: %s (%s)" % (i, word_sense.name(), get_definition(word_sense)))
  print("Hypernyms:")
  hypernyms = word_sense.hypernyms()
  while len(hypernyms) >0:
    print("%s\tis a\t%s" % (word_sense.name(), hypernyms[0].name()))
    word_sense = hypernyms[0]
    hypernyms = word_sense.hypernyms()


Sense 0: bug.n.01 (general term for any insect or similar creeping or crawling invertebrate)
Hypernyms:
bug.n.01	is a	insect.n.01
insect.n.01	is a	arthropod.n.01
arthropod.n.01	is a	invertebrate.n.01
invertebrate.n.01	is a	animal.n.01
animal.n.01	is a	organism.n.01
organism.n.01	is a	living_thing.n.01
living_thing.n.01	is a	whole.n.02
whole.n.02	is a	object.n.01
object.n.01	is a	physical_entity.n.01
physical_entity.n.01	is a	entity.n.01

Sense 1: bug.n.02 (a fault or defect in a computer program, system, or machine)
Hypernyms:
bug.n.02	is a	defect.n.03
defect.n.03	is a	imperfection.n.01
imperfection.n.01	is a	state.n.02
state.n.02	is a	attribute.n.02
attribute.n.02	is a	abstraction.n.06
abstraction.n.06	is a	entity.n.01

Sense 2: bug.n.03 (a small hidden microphone; for listening secretly)
Hypernyms:
bug.n.03	is a	microphone.n.01
microphone.n.01	is a	electro-acoustic_transducer.n.01
electro-acoustic_transducer.n.01	is a	transducer.n.01
transducer.n.01	is a	electrical_device.n.01
elect

## Game Idea
One of the tricky things about creating a text adventure game is anticipating the many different ways that a player might write a command.

If you program the game to understand a command like _give fish to troll_ and the player types in _feed fish to troll_, then a simple parser will fail to undertand the command.

Here, we show you how to use WordNet to expand out the set of commands that you've programed into the game, with the goal of being able to recognize more varied input from the player.  

Instead of just one _give fish to troll_ command, we'll enumerate thouands of alternatives like

* _serve salmon to monster_
* _serve up food to monster_
* _feed sea trout to troll_
* _supply smoked salmon to mythical creature_

In [0]:

commands = [
	'give fish to troll',
]

commands = [
	'wear crown',
	'smell rose',
	'eat fish',
	'light lamp',
	'give fish to troll',
	'propose to the princess',
	'go north',
]



## Manually Annotate Senses and Hypernyms/Hyponyms

Below is some code that will help you manually annotate the word sense of each word in your list of commands, and confirm which hypernyms and hyponyms are reasonable substitutes that should be recognized if a player types them instead of our command word.

Here are some helper functions for you. You can just run this cell instead of reading through the functions in detail if you want. 

In [0]:
def annotate_synsets(sentences):
  """This function queries WordNet for each word in a list of sentences,
     and asks the user to input a number corresponding to the synset."""

  word_senses = {}
  # Cached selections maps from word string to the previous
  # selection for this word (an integer)
  cached_selections = {}

  for i, sent in enumerate(sentences):
    words = word_tokenize(sent.lower())

    for word in words:
      sysnsets = wn.synsets(word)
      if len(sysnsets) != 0:
        selection = select_synset(sent, word, sysnsets, cached_selections)
        if selection != None:
          cached_selections[word] = selection
          if selection < len(sysnsets):
            s = sysnsets[selection]
            word_senses[word] = s.name()
  return word_senses


def select_synset(sent, word, sysnsets, cached_selections):
  """Ask the user to select which sense of the word  
     is being used in this sentence."""
  print(sent)
  print(word.upper())

  prev_selection = -1
  if word in cached_selections:
    prev_selection = cached_selections[word]

  for choice, s in enumerate(sysnsets):
    if choice == prev_selection:
      print("*** ", end = '')
    print("%d) %s - %s" % (choice, s.name(), s.definition()))

  choice += 1
  if choice == prev_selection:
    print("*** ", end = '')
  print("%d None of these." % choice)

  selection = -1
  while selection == -1:
    try:
      user_input = input(">")
      if user_input.strip() == 'x':
        # The user can press 'x' to exit.
        return None
      if user_input.strip() == '' and prev_selection > -1:
        # The user can press retrun to confirm the previous selection.
        return prev_selection
      selection = int(user_input)
    except:
      selection = -1
    if selection < 0 or selection > len(sysnsets):
      print("Please select a number between 0-%d, or type 'x' to exit" % len(sysnsets))
      if prev_selection > -1:
        print("You can also press return to confirm the previous selection (marked by ***).")
    else:
      return selection


def confirm_hyponyms(word, sysnset, do_hypernyms_instead=False):
  """Ask the user to confirm which of the hyponyms are applicable 
     for this sentence."""
  print(word.upper())

  confirmed = []
  if do_hypernyms_instead:
    unconfirmed = sysnset.hypernyms()
  else:
    unconfirmed = sysnset.hyponyms()

  while len(unconfirmed) > 0:
    s = unconfirmed.pop(0)
    print("Is %s an appropriate substitute for %s? (y/n)" % (s.name(), word))
    print("It means:", s.definition())
    print("Synonyms are:", get_synonyms(s))
    user_input = ''
    while user_input == '':
      user_input = input(">")
      user_input = user_input.strip()
      if user_input == 'y' or user_input == 'yes':
        confirmed.append(s.name())
        if do_hypernyms_instead:
          unconfirmed.extend(s.hypernyms())
        else:
          unconfirmed.extend(s.hyponyms())
        
      elif user_input == 'n' or user_input == 'no':
        pass
      elif user_input == 'x':
        # The user can press 'x' to exit.
        return confirmed
      else:
        print("Please type 'yes' or 'no' or 'x' to stop confirming for this word")
        user_input = ''
  return confirmed

# Save your annotations to a file, so that you can submit them with your homework.
def save_to_drive(word_senses, confirmed_hyponyms, confirmed_hypernyms):
  import json
  from google.colab import drive
  drive.mount('/content/drive/')

  output_file = '/content/drive/My Drive/word-sense-annotations.json'
  output_json = {}
  output_json['senses'] = word_senses
  output_json['hyponyms'] = confirmed_hyponyms
  output_json['hypernyms'] = confirmed_hypernyms

  with open(output_file, 'w') as write_file:
    write_file.write(json.dumps(output_json, sort_keys=True, indent=4))
    write_file.write('\n')



Run this part when you're ready to start annotating the words in the commands. I estimate that it will take about 10 minutes per command.

In [0]:
word_senses = annotate_synsets(commands)
confirmed_hyponyms = {}
confirmed_hypernyms = {}
for word in word_senses:
  print("First, pick the word sense for the word '%s'" % word)
  print("==============")
  word_sense = wn.synset(word_senses[word])
  print("\nNext, pick which hypernyms of %s we should allow players to use." % word_sense.name())
  print("==============")
  confirmed_hypernyms[word] = confirm_hyponyms(word, word_sense, do_hypernyms_instead=True)
  print("\Finally, pick which hyponyms of %s we should allow players to use." % word_sense.name())
  print("==============")  
  confirmed_hyponyms[word] = confirm_hyponyms(word, word_sense)


print("You're done annotating!  Save your annotation to your Google drive.")
print("You need to paste in a confirmation code to allow Colab to have access.")
print("We'll create a file called 'word-sense-annotations.json' for you to turn in.")
print("==============")
save_to_drive(word_senses, confirmed_hyponyms, confirmed_hypernyms)



wear crown
WEAR
0) wear.n.01 - impairment resulting from long use
1) clothing.n.01 - a covering designed to be worn on a person's body
2) wear.n.03 - the act of having on your person as a covering or adornment
3) wear.v.01 - be dressed in
4) wear.v.02 - have on one's person
5) wear.v.03 - have in one's aspect; wear an expression of one's attitude or personality
6) wear.v.04 - deteriorate through use or stress
7) wear.v.05 - have or show an appearance of
8) wear.v.06 - last and be usable
9) break.v.42 - go to pieces
10) tire.v.02 - exhaust or get tired through overuse or great strain or stress
11) wear.v.09 - put clothing on one's body
12 None of these.


## Look Over Your Annotations

In [0]:
for word in word_senses:
  print('\n', word.upper())
  word_sense = wn.synset(word_senses[word])
  print('Synonyms:\t', get_synonyms(word_sense))
  print('Hypernyms:', )
  for hypernym in confirmed_hypernyms[word]:
    print('\t', get_synonyms(wn.synset(hypernym)))

  print('Hyponyms:', )
  hyponyms = confirmed_hyponyms[word]
  for hyponym in hyponyms:
    print('\t', get_synonyms(wn.synset(hyponym)))

## Enumerate Alternatives Wordings of Commands

In [0]:
import itertools

def get_alternatives(word, word_senses, confirmed_hypernyms, confirmed_hyponyms):
  alternatives = []
  if not word in word_senses:
    alternatives.append(word)
    return alternatives
  word_sense = wn.synset(word_senses[word])
  alternatives.extend(get_synonyms(word_sense))
  for hypernym in confirmed_hypernyms[word]:
    alternatives.extend(get_synonyms(wn.synset(hypernym)))
  for hyponym in confirmed_hyponyms[word]:
    alternatives.extend(get_synonyms(wn.synset(hyponym)))
  return alternatives

def enumerate_alternatives(sentence, word_senses, confirmed_hypernyms, confirmed_hyponyms):
  words = word_tokenize(sentence.lower())
  # a list of lists
  alternatives_per_word = []
  for word in words:
    alternatives = get_alternatives(word, word_senses, confirmed_hypernyms, confirmed_hyponyms)
    alternatives_per_word.append(alternatives)
  
  alternative_to_original = {}
  # all combinations of a list of lists
  for words in list(itertools.product(*alternatives_per_word)):
    alt_sent = " ".join(words)
    alternative_to_original[alt_sent] = sentence
  return alternative_to_original


# alternative_commands is a dictionary that maps 
# the new commands onto the original ones.
alternative_commands = {}
for command in commands:
  alternative_commands.update(enumerate_alternatives(command, 
                                                     word_senses, 
                                                     confirmed_hypernyms, 
                                                     confirmed_hyponyms))

for alt_sent in alternative_commands:
  print("%s ==> %s" % (alt_sent, alternative_to_original[alt_sent]))
print("Congratulations you can now handle %d commands instead of just %d!" % 
      (len(alternative_to_original.keys()), len(commands)))

In [0]:
word = "great"
for s in wn.synsets(word):
  print('The name of the synset is:', s.name())
  print('Synonyms in this synsets are:')
  for lemma in s.lemmas():
    print('\t', lemma.name())
  print('The definition is:', s.definition())
  if s.examples():
    print('Example uses are:\n\t', '\n\t'.join(s.examples()))
  if s.part_holonyms():
    print("Part holonyms:", s.part_holonyms())
  if s.part_meronyms():
    print("Part meronyms:", s.part_meronyms())
  if s.member_holonyms():
    print("Member holonyms:", s.member_holonyms())
  if s.member_meronyms():
    print("Member meronyms:", s.member_meronyms())
  print('\n')

In [0]:
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()


synset = wn.synsets('dog')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')

synset = wn.synsets('branch')[1]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')


synset = wn.synsets('ogre')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')



synset = wn.synsets('fears')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep ='\n')
print('\n')

synset = wn.synsets('height')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')




In [0]:
"""Morphy performs morphological analysis to get the base form of a word.
   It uses a combination of inflectional ending rules and exception lists
  to handle a variety of different possibilities."""

print(wn.morphy('dogs'))
print(wn.morphy('dogged'))
print(wn.morphy('wolves'))
print(wn.morphy('corpora'))
print(wn.morphy('skiing', wn.VERB))


## Visualizing WordNet

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue
from nltk.corpus import wordnet as wn


In [0]:
import collections

class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()

In [0]:
def DFS(graph, word="bug", cutoff=100):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""

  synsets = wn.synsets(word)
  frontier = Queue()
  visited = {}

  for i in range(len(synsets)):
    synset = synsets[i]
    frontier.put(synset)
    visited[synset] = True

  num = 0

  while not frontier.empty():
    synset = frontier.get()
    visited[synset] = True

    num += 1
    if num > cutoff:
      return

    name = synset.name()
    # Create a new node in the graph for this location
    graph.node(name, label=name)
    for hypernym in synset.hypernyms():
      if not hypernym in visited:
        frontier.put(hypernym)
        graph.edge(hypernym.name(), name)

    for hyponym in synset.hyponyms():
      if not hyponym in visited:
        frontier.put(hyponym)
        graph.edge(name, hyponym.name())

  


graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
DFS(graph, cutoff=1000)
graph


In [0]:
#!sudo pip3 install nltk 
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Polysemy
Words with multiple meanings are called _polysemous_ words.  An example of a polysemous word is the noun _bug_ which can mean 
1. an insect
2. a virus or microbe that makes you sick
3. an error in your computer program
4. a covert listening device

WordNet oraganizes word senses into a structure called _synsets_. Each word can have multiple synsets, and each synset represents a different meaning of the word.

In [0]:
sentences = [
	'You are standing on one side of a drawbridge leading to the castle.',
	'You are climbing the stairs to the tower. There is a locked door here.',
	'You are climbing the stairs down to the dungeon.',
	'You are walking along a winding path. There is a tall tree here.',
	'You are standing on a lush garden path. There is a cottage here.',
	'a guard carrying a sword and a key',
	'HE LOOKS AT YOU SUSPICIOUSLY.',
	'A ghost with bony, claw-like fingers and who is wearing a crown.',
	'a small fishing pond',
	'THE CANDLE IS COVERED IN STARGE RUNES.  IT IS EMITTING A STRANGE, ACRID SMELLING SMOKE.',
	'an unconscious guard is slumpped against the wall',
	'unlock door',
	'catch fish with pole',
	'smell rose',
	'eat fish',
	'light lamp',
	'read runes',
	'wear crown',
	'propose to the princess',
	'kiss princess',
]

In [0]:

def annotate_synsets(sentences):
  """This function queries WordNet for each word in a list of sentences,
     and asks the user to input a number corresponding to the synset."""

  for sent in sentences:
    print(sent)
    for word in word_tokenize(sent.lower()):
      print(word.upper())

      sysnsets = wn.synsets(word)
      if len(sysnsets) == 0:
        return None
      for i, s in enumerate(sysnsets):
        print(i, ")", s.name(), s.definition())
      selection = -1
      while selection == -1:
        try:
          user_input = input(">")
          if user_input.strip() == 'x':
            return None
          selection = int(user_input)
        except:
          selection = -1
        if selection < 0 or selection > len(sysnsets):
          print("please select a number between 0-%d, or type 'x' to exit")
        else:
          return sysnsets[selection]

def print_synonyms(s):
  for lemma in s.lemmas():
    print(lemma.name())

annotate_synsets(sentences)






You are standing on one side of a drawbridge leading to the castle.
YOU


In [0]:
word = "great"
for s in wn.synsets(word):
  print('The name of the synset is:', s.name())
  print('Synonyms in this synsets are:')
  for lemma in s.lemmas():
    print('\t', lemma.name())
  print('The definition is:', s.definition())
  if s.examples():
    print('Example uses are:\n\t', '\n\t'.join(s.examples()))
  if s.part_holonyms():
    print("Part holonyms:", s.part_holonyms())
  if s.part_meronyms():
    print("Part meronyms:", s.part_meronyms())
  if s.member_holonyms():
    print("Member holonyms:", s.member_holonyms())
  if s.member_meronyms():
    print("Member meronyms:", s.member_meronyms())
  print('\n')

The name of the synset is: great.n.01
Synonyms in this synsets are:
	 great
The definition is: a person who has achieved distinction and honor in some field
Example uses are:
	 he is one of the greats of American music


The name of the synset is: great.s.01
Synonyms in this synsets are:
	 great
The definition is: relatively large in size or number or extent; larger than others of its kind
Example uses are:
	 a great juicy steak
	a great multitude
	the great auk
	a great old oak
	a great ocean liner
	a great delay


The name of the synset is: great.s.02
Synonyms in this synsets are:
	 great
	 outstanding
The definition is: of major significance or importance
Example uses are:
	 a great work of art
	Einstein was one of the outstanding figures of the 20th centurey


The name of the synset is: great.s.03
Synonyms in this synsets are:
	 great
The definition is: remarkable or out of the ordinary in degree or magnitude or effect
Example uses are:
	 a great crisis
	had a great stake in the ou

In [0]:
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()


synset = wn.synsets('dog')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')

synset = wn.synsets('branch')[1]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')


synset = wn.synsets('ogre')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')



synset = wn.synsets('fears')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep ='\n')
print('\n')

synset = wn.synsets('height')[0]
print(synset)
print(*list(synset.closure(hyper, depth=15)), sep='\n')
print('\n')




Synset('dog.n.01')
Synset('canine.n.02')
Synset('domestic_animal.n.01')
Synset('carnivore.n.01')
Synset('animal.n.01')
Synset('placental.n.01')
Synset('organism.n.01')
Synset('mammal.n.01')
Synset('living_thing.n.01')
Synset('vertebrate.n.01')
Synset('whole.n.02')
Synset('chordate.n.01')
Synset('object.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


Synset('branch.n.02')
Synset('stalk.n.02')
Synset('plant_organ.n.01')
Synset('plant_part.n.01')
Synset('natural_object.n.01')
Synset('whole.n.02')
Synset('object.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


Synset('monster.n.04')
Synset('unpleasant_person.n.01')
Synset('unwelcome_person.n.01')
Synset('person.n.01')
Synset('causal_agent.n.01')
Synset('organism.n.01')
Synset('physical_entity.n.01')
Synset('living_thing.n.01')
Synset('entity.n.01')
Synset('whole.n.02')
Synset('object.n.01')


Synset('fear.n.01')
Synset('emotion.n.01')
Synset('feeling.n.01')
Synset('state.n.02')
Synset('attribute.n.02')
Synset('ab

In [0]:
"""Morphy performs morphological analysis to get the base form of a word.
   It uses a combination of inflectional ending rules and exception lists
  to handle a variety of different possibilities."""

print(wn.morphy('dogs'))
print(wn.morphy('dogged'))
print(wn.morphy('wolves'))
print(wn.morphy('corpora'))
print(wn.morphy('skiing', wn.VERB))


dog
dog
wolf
corpus
ski


## Visualizing WordNet

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue
from nltk.corpus import wordnet as wn


In [0]:
import collections

class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()

In [0]:
def DFS(graph, word="bug", cutoff=100):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""

  synsets = wn.synsets(word)
  frontier = Queue()
  visited = {}

  for i in range(len(synsets)):
    synset = synsets[i]
    frontier.put(synset)
    visited[synset] = True

  num = 0

  while not frontier.empty():
    synset = frontier.get()
    visited[synset] = True

    num += 1
    if num > cutoff:
      return

    name = synset.name()
    # Create a new node in the graph for this location
    graph.node(name, label=name)
    for hypernym in synset.hypernyms():
      if not hypernym in visited:
        frontier.put(hypernym)
        graph.edge(hypernym.name(), name)

    for hyponym in synset.hyponyms():
      if not hyponym in visited:
        frontier.put(hyponym)
        graph.edge(name, hyponym.name())

  


graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
DFS(graph, cutoff=1000)
graph


# ConceptNet

## TODO

In [0]:
pip install ConceptNet

# Dependendency Parsing

A dependency parser analyzes the grammatical structure of a sentence, comptuting parts-of-speech and establishing relationships between the words. It outputs a tree structure of the sentence.

For example, consider the command:
```
Take the apple from the table and eat it.
```

The parse for this command would look like:
```
(ROOT
  (S
    (VP
      (VP (VB Take)
        (NP (DT the) (NN apple))
        (PP (IN from)
          (NP (DT the) (NN table))))
      (CC and)
      (VP (VB eat)
        (NP (PRP it))))
    (. .)))
```
Furthermore, "apple" and "it" are both recognized as direct objects being acted upon by the verbs "take" and "eat' respectively.

You can also visualize this tree structure using AllenNLP's very cool tool located [here](https://demo.allennlp.org/dependency-parsing).

<div>
<img src="https://github.com/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/homeworks/nlp-for-text-adventures/parse.png?raw=true" width="500"/>
</div>

**Game Idea:** Instead of hardcoding "verb object" into your parser, use the dependency parse of the command to figure out which words are the verb and direct object. If you want to get even fancier, break the sentence into clauses, and extract the vern and direct object from each of them.

In [0]:
!pip install allennlp

In [33]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/biaffine-dependency-parser-ptb-2018.08.23.tar.gz")

Did not use initialization regex that was passed: .*bias_hh.*
Did not use initialization regex that was passed: .*weight_hh.*
Did not use initialization regex that was passed: .*bias_ih.*
Did not use initialization regex that was passed: .*weight_ih.*


In [39]:
predictor.predict(
  sentence="Take the apple from the table and eat it."
)

{'arc_loss': 1.0490381328054355e-06,
 'hierplane_tree': {'linkToPosition': {'acomp': 'right',
   'ccomp': 'right',
   'csubj': 'left',
   'csubjpass': 'left',
   'dobj': 'right',
   'iobj': 'right',
   'nsubj': 'left',
   'nsubjpass': 'left',
   'pcomp': 'right',
   'pobj': 'right',
   'xcomp': 'right'},
  'nodeTypeToStyle': {'acomp': ['color2'],
   'advcl': ['color3'],
   'advmod': ['color4'],
   'amod': ['color4'],
   'appos': ['color4'],
   'aux': ['color3'],
   'cc': ['color3'],
   'ccomp': ['color2'],
   'conj': ['color3'],
   'cop': ['color3'],
   'csubj': ['color1'],
   'csubjpass': ['color1'],
   'dep': ['color5', 'strong'],
   'det': ['color3'],
   'discourse': ['color3'],
   'dobj': ['color2'],
   'expletive': ['color3'],
   'infmod': ['color4'],
   'iobj': ['color2'],
   'mark': ['color2'],
   'mod': ['color4'],
   'neg': ['color0'],
   'nn': ['color4'],
   'npadvmod': ['color4'],
   'nsubj': ['color1'],
   'nsubjpass': ['color1'],
   'number': ['color3'],
   'pcomp': ['colo

In [38]:
prediction = predictor.predict(
  sentence="Take the apple from the table and eat it."
)

tuples = zip(prediction['predicted_dependencies'], prediction['predicted_heads'])
for dependency, head_index in tuples:
  if dependency == 'dobj':
    print(prediction['hierplane_tree'][head_index])


KeyError: ignored

# Sentiment Analysis
A common classification problem is detecting whether a text has positive or negartive sentiment. 

A library called [TextBlob](https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis) provides a pre-trained sentiment model that you can use.

**Game Idea:**
There are two guards to get past, one that only lets you through if you insult him, another only if you complement him.

In [0]:
import nltk
nltk.download('punkt')
from textblob import TextBlob

text = '''
I enjoyed my stay tremendously; what incredible service!
The castle was absolutely incredible to visit, especially its voluminous dungeons.
My expectations were high, but the castle ended up being only so-so.
You're a despicable excuse for a guard; it's a wonder you were hired.
'''

blob = TextBlob(text)

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0.611111111111111
0.45
0.08
-0.05


# Word Embeddings
Word embeddings offer a way to compare the similarity of two words. Words are mapped to a vector space where semantically similar words have a low Euclidean distance between. This allows us to make statements like:

$$
\text{"Apple" is closer to "pear" than it is to "kitten" because $\|$emb(apple) - emb(pear)$\| < \|$emb(apple) - emb(kitten)$\|$.}
$$

A Python library called [Magnitude](https://github.com/plasticityai/magnitude/) facilitates easy access to word embeddings. We show a couple simple examples below, but their README has many more.

**Game Idea:**
When a user types "take object," check whether any objects close (in vector space) to "object" are currently in the room.

In [0]:
!pip install pymagnitude

# To avoid a big download, we retrive the 50-dimensional embeddings.
# However, you'll want to grab the 300-dimensional embeddings if you 
# choose to use these in your game.
#!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.300d.magnitude
!wget -N http://magnitude.plasticity.ai/glove/light/glove.6B.50d.magnitude

     |████████████████████████████████| 5.4MB 4.0MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918205 sha256=0c4f04909e877312aa5d077dd0c44de8f3c136f93cad04171a4821500afbb10f
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude
--2020-01-21 21:05:16--  http://magnitude.plasticity.ai/glove/light/glove.6B.50d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.216.142.243
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.216.142.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102670336 (98M) [binary/octet-stream]
Saving to: ‘glove.6B.50d.magnitude’

glove.6B.50d.magnit 100%[===================>]  97.91M  36.8MB/s    in 2.7s    

2020-01-21 21:05:19 (36.8 MB/s) - ‘glove.6B.50d.magnitude’ saved [102670336/102670336]



In [0]:
from pymagnitude import *
import numpy as np

word2vec = Magnitude("glove.6B.50d.magnitude")

In [0]:
kitten = word2vec.query("kitten")
pear = word2vec.query("pear")
apple = word2vec.query("apple")

dist = np.sqrt(np.sum(np.power(kitten - pear, 2)))
print('The distance between "kitten" and "pear" is: %f' % dist)
print('It can also be computed as: %f' % word2vec.distance("kitten", "pear"))

print('The distance between "apple" and "pear" is: %f' %
      word2vec.distance("apple", "pear"))

The distance between "kitten" and "pear" is: 1.285782
It can also be computed as: 1.285782
The distance between "apple" and "pear" is: 0.955566


# Predicting Word Concreteness
It's possible from a word's embedding to prdict how concrete the word is.

Concreteness is a measure of how readily the concerpt repreesented by a word can be seen, smelled, heard, or felt. 

If a concept can be readily perceived by the senses then is is very concrete. If a concept cannot be perceived, then it is the opposite of concrete--abstract.

**Game Idea:** TODO

In [0]:
!wget -N http://crr.ugent.be/papers/Concreteness_ratings_Brysbaert_et_al_BRM.txt

import csv
from tqdm import tqdm
from zlib import crc32

import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPRegressor

import scipy.stats

--2020-01-21 21:09:22--  http://crr.ugent.be/papers/Concreteness_ratings_Brysbaert_et_al_BRM.txt
Resolving crr.ugent.be (crr.ugent.be)... 157.193.153.14
Connecting to crr.ugent.be (crr.ugent.be)|157.193.153.14|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘Concreteness_ratings_Brysbaert_et_al_BRM.txt’ not modified on server. Omitting download.



#### Helper methods

In [0]:
def read_in_data(file_path, word2vec):
  words = []
  concs = []
  embs = []
  with open(file_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t',)
    for row in tqdm(reader):
      conc = float(row['Conc.M'])
      word = row['Word']
      if conc != 0:
        # 0 means there was not enough interannotator agreement for them to
        # include the score.

        word = word.replace(' ', '-').lower()
        if word in word2vec:
          # For now, skip words not in the embedding file. 
          embs.append(word2vec.query(word))
          words.append(word)
          concs.append(conc)
  return words, concs, embs

def floathash(b):
  return float(crc32(b.encode('utf-8')) & 0xffffffff) / 2**32

def create_split(words, concs, embs, train_prob = 0.9):
  val_words = []
  val_concs = []
  val_embs = []

  train_words = []
  train_concs = []
  train_embs = []

  for word, conc, emb in zip(words, concs, embs):
    if floathash(word) <= train_prob:
      train_words.append(word)
      train_concs.append(conc)
      train_embs.append(emb)
    else:
      val_words.append(word)
      val_concs.append(conc)
      val_embs.append(emb)
  return train_words, train_concs, train_embs, val_words, val_concs, val_embs 

def crush_scores(scores):
  """Turn 1-5 scores to 0-1 scale."""
  return [(s - 1) / 4.0 for s in scores]

def train_model(train_embs, train_concs, val_embs, val_concs, method='linear', normalize=False):
  print('Training with method %s, %s' % (method, '[0,1]' if normalize else '[1,4]'))
  if normalize:
    val_concs = crush_scores(val_concs)
    train_concs = crush_scores(train_concs)    
  if method == 'linear':
    model = LinearRegression()
  elif method == '2mlp':
    model = MLPRegressor(hidden_layer_sizes=[64,32])
  else:
    raise ValueError('Unsupported method')

  model = model.fit(train_embs, train_concs)
  print('Train correlation: ')
  print(scipy.stats.pearsonr(model.predict(train_embs), train_concs))
  
  print('Val correlation: ')
  print(scipy.stats.pearsonr(model.predict(val_embs), val_concs))
  
  print('')
  return model
  

#### Read in data and train small model.

In [0]:
words, concs, embs = read_in_data('Concreteness_ratings_Brysbaert_et_al_BRM.txt', word2vec)

train_words, train_concs, train_embs, val_words, val_concs, val_embs = create_split(words, concs, embs, 0.95)
print('Train set size: %d' % len(train_words))
print('Val set size: %d' % len(val_words))

model = train_model(train_embs, train_concs, val_embs, val_concs, '2mlp', True)

39954it [00:04, 9977.64it/s] 


Train set size: 30492
Val set size: 1602
Training with method 2mlp, [0,1]
Train correlation: 
(0.8549550047245305, 0.0)
Val correlation: 
(0.82438336559675, 0.0)



### Some predictions for words not in train set

In [0]:
print('archetype' in train_words)
print(model.predict([word2vec.query('archetype')]))

print('pigtailed' in train_words)
print(model.predict([word2vec.query('pigtailed')]))

print('determination' in train_words)
print(model.predict([word2vec.query('determination')]))

print('whirlpool' in train_words)
print(model.predict([word2vec.query('whirlpool')]))

False
[0.28465186]
False
[0.80104775]
False
[0.81268915]
False
[0.25966937]


In [0]:
print(val_words)

['waterbed', 'pregnant', 'pigtailed', 'beardless', 'redheaded', 'toothless', 'unshaved', 'red', 'seamed', 'scaly', 'chalky', 'corded', 'perforated', 'blooming', 'earless', 'spiky', 'steroid', 'unbuckled', 'stormy', 'phosphorous', 'windowless', 'antiseptic', 'foggy', 'pale', 'fluorescent', 'chubby', 'shaven', 'tattered', 'winged', 'deformed', 'gooey', 'lukewarm', 'televised', 'spinal', 'drinkable', 'upward', 'prostrate', 'unsigned', 'bubbly', 'ratty', 'circulatory', 'unburied', 'unburned', 'granular', 'jellylike', 'confectionary', 'overgrown', 'periodontal', 'slobbery', 'leaner', 'uncompleted', 'oozy', 'yearly', 'tropic', 'sour', 'tidal', 'lanky', 'holographic', 'motherless', 'dual', 'inflatable', 'weathered', 'purebred', 'submersible', 'abrasive', 'unpainted', 'watertight', 'weary', 'easternmost', 'fertile', 'seasonal', 'measurable', 'northwestern', 'whirly', 'retail', 'nippy', 'unscreened', 'cheesy', 'sedimentary', 'sober', 'cloistered', 'whispery', 'ailing', 'trashy', 'glandular', 'w

# BERT Contextual Word Embeddings
One issue with word embeddings is that they don't handle ambiguity. If I say the word "bat", do you picture baseball or a cute flying mammal?  Word2vec would end up picking a vector somewhere in between the two.

Contextual word embeddings are word embeddings that vary based on the context in which a word is being used.

Consider the following sentences.
```
1) The bat comes out at night to eat mosquitoes.
2) The swallow flitted from branch to branch, eating mosquitoes.
3) The player dropped the bat and sprinted past first base.
```

With contextual word embeddings, the embedding of "bat" in (1) will end up being close to the embedding for "swallow" in  (2) than the embedding of "bat" in (3).

BERT is a neural network trained to produce one embedding per token in the input

**Game Idea:** TODO

In [11]:
!pip install transformers

import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 450kB 2.7MB/s 
     |████████████████████████████████| 870kB 21.2MB/s 
     |████████████████████████████████| 1.0MB 37.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=7353fc2100dc88472beb6307a9014675ed2c18c10aee7366ee332d7f4f0d58e1
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


#### Helper methods

In [0]:
def get_tokens_and_embeddings(text):
  inputs_ids = tokenizer.encode(text)
  input_ids = torch.tensor(inputs_ids).unsqueeze(0)  # Batch size 1

  token_embeddings, merged_embedding = model(input_ids)

  # Remove the embeddings in the first and last positions
  # which are the [CLS] and [SEP] tokens.
  token_embeddings = token_embeddings.squeeze()[1:-1, :]
  return token_embeddings.detach().numpy()

def token_indexes_for_word(tokens, word):
  """Returns the token indexes corresponding to the specified word."""
  ids = tokenizer.convert_tokens_to_ids(tokens)

  word_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
  word_len = len(word_ids)

  for i in range(len(tokens) - word_len):
    if np.all(np.equal(ids[i:(i+word_len)], word_ids)):
      return list(range(i, i+word_len))
  return None

#### Computing a word embedding

In [25]:
# Since BERT uses a subword vocabulary can take up multiple tokens.
# This can be seen in the word "mosquitoes" in the following sentence.
sentence = "The bat comes out at night to eat mosquitoes."
embeddings = get_tokens_and_embeddings(sentence)
tokens = tokenizer.tokenize(sentence)
mosquitoes_indices = token_indexes_for_word(tokens, "mosquitoes")
print(sentence)
print(tokens)
print("'mosquitoes' is in token positions: %s" % str(mosquitoes_indices))

# For 'mosquitoes' and other multi-token words, a single embedding for the word
# can be computed by simply taking the embedding of the first token of the word.
# Another option is to take the mean over all of the constituent token
# embeddings.
mosquitoes_embedding = embeddings[mosquitoes_indices[0], :]
alternative_mosquitoes_embedding = np.mean(embeddings[mosquitoes_indices, :], axis=0)
print(mosquitoes_embedding.shape)
print(alternative_mosquitoes_embedding.shape)

The bat comes out at night to eat mosquitoes.
['the', 'bat', 'comes', 'out', 'at', 'night', 'to', 'eat', 'mosquito', '##es', '.']
'mosquitoes' is in token positions: [8, 9]
(768,)
(768,)


#### Comparing contextual word embeddings

In [30]:
sentence = "The bat comes out at night to eat mosquitoes."
embeddings = get_tokens_and_embeddings(sentence)
animalbat_index = token_indexes_for_word(tokens, "bat")[0]
animalbat_embedding = embeddings[animalbat_index, :]

sentence = "The swallow flitted from branch to branch, eating mosquitoes."
embeddings = get_tokens_and_embeddings(sentence)
swallow_index = token_indexes_for_word(tokens, "bat")[0]
swallow_embedding = embeddings[swallow_index, :]

sentence = "The player dropped the bat and sprinted past first base."
embeddings = get_tokens_and_embeddings(sentence)
baseballbat_index = token_indexes_for_word(tokens, "bat")[0]
baseballbat_embedding = embeddings[baseballbat_index, :]

print('Distance between a swallow and an animal bat: %f' %
      cosine(animalbat_embedding, swallow_embedding))
print('Distance between an animal bat and a baseball bat: %f' %
      cosine(animalbat_embedding, baseballbat_embedding))
print('Distance between a swallow and a baseball bat: %f' %
      cosine(swallow_embedding, baseballbat_embedding))

Distance between a swallow and an animal bat: 0.346092
Distance between an animal bat and a baseball bat: 0.666941
Distance between a swallow and a baseball bat: 0.706544
